# Style Transfer

**Mark Wilber**

Uses style transfer technique to synthesize paintings in the style of Bonnie Wilber from photographs

Borrows heavily from Walid Ahmad's [Making AI Art with Style Transfer using Keras](https://medium.com/mlreview/making-ai-art-with-style-transfer-using-keras-8bb5fa44b216)

<font color='darkgreen'>**As thise notebook is lengthy, readers will find it much easier to navigate with [Jupyter Nbextensions](https://github.com/ipython-contrib/jupyter_contrib_nbextensions) installed, and Table of Contents (2) selected:**</font>

## Preliminaries

**Next two lines are useful in the event of external code changes.**

In [ ]:
%load_ext autoreload
%autoreload 2

### Python imports

**Next two lines are for pretty output for all prints in a Pandas cell, not just the last.**

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**`DataSci` contains generally helpful data science stuff, while `plotHelpers` includes plot functions specifically.**

In [ ]:
import sys
# sys.path.append('/home/wilber/work/Mlib')
# from utility import DataSci as util
# import plotHelpers as ph

In [ ]:
from time import time, asctime, gmtime, localtime
print(asctime(gmtime()))

t0 = time()

# from platform import node
import os
from os.path import exists
# import shutil
# from glob import glob
# from random import random
# from collections import Counter, OrderedDict
# import gc		# garbage collection module
# import pprint
# import pickle
# import timeit

print("Python version: ", sys.version_info[:])
print("Un-versioned imports:\n")
if 'sys' in sys.modules:
    print('sys', end="")
if 'utility' in sys.modules:
    print(', utility', end="")
if 'plotHelpers' in sys.modules:
    print(', plotHelpers', end="")
if 'platform' in sys.modules:
    print(', platform', end="")
if 'os' in sys.modules:
    print(', os', end="")
if 'os.path' in sys.modules:
    print(', os.path', end="")
if 'shutil' in sys.modules:
    print(', shutil', end="")
if 'glob' in sys.modules:
    print(', glob', end="")
if 'random' in sys.modules:
    print(', random', end="")
if 'collections' in sys.modules:
    print(', collections', end="")
if 'gc' in sys.modules:
    print(', gc', end="")
if 'pprint' in sys.modules:
    print(', pprint', end="")
if 'pickle' in sys.modules:
    print(', pickle', end="")
if 'timeit' in sys.modules:
    print(', timeit', end="")

import dateutil as du
# from dateutil.parser import parse
import numpy as np
# import pandas as pd
# import pyreadr

from scipy import __version__ as scVersion
# import scipy.sparse as sp
from scipy.optimize import fmin_l_bfgs_b

import tensorflow as tf
from keras import backend as K, __version__ as kerVersion
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import VGG16
from keras.layers import Input

from PIL import Image, __version__ as pilVersion

# from sklearn import __version__ as skVersion
# from sklearn.feature_extraction import text
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_selection import chi2
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.svm import LinearSVC, SVC
# from sklearn.naive_bayes import ComplementNB

# from joblib import __version__ as jlVersion
# from joblib import dump, load

# import seaborn as sns
# import colorcet as cc
from matplotlib import __version__ as mpVersion
import matplotlib.pyplot as plt

print("\n")
if 'dateutil' in sys.modules:
    print("dateutil: {0}".format(du.__version__), end="\t")
if 'numpy' in sys.modules:
    print("numpy: {0}".format(np.__version__), end="\t")
# if 'pandas' in sys.modules:
#     print("pandas: {0}".format(pd.__version__), end="\t")
if 'pyreader' in sys.modules:
    print("pyreader: {0}".format(pyreader.__version__), end="\t")
if 'scipy' in sys.modules:
    print("scipy: {0}".format(scVersion), end="\t")
if 'tensorflow' in sys.modules:
    print("tensorflow: {0}".format(tf.__version__), end="\t")
if 'keras' in sys.modules:
    print("keras: {0}".format(kerVersion), end="\t")
if 'PIL' in sys.modules:
    print("PIL: {0}".format(pilVersion), end="\t")
if 'sklearn' in sys.modules:
    print("sklearn: {0}".format(skVersion), end="\t")
if 'joblib' in sys.modules:
    print("joblib: {0}".format(jlVersion), end="\t")
if 'seaborn' in sys.modules:
    print("seaborn: {0}".format(sns.__version__), end="\t")
if 'colorcet' in sys.modules:
    print("colorcet: {0}".format(cc.__version__), end="\t")
if 'matplotlib' in sys.modules:
    print("matplotlib: {0}".format(mpVersion), end="\t")
# if '' in sys.modules:
#     print(": {0}".format(.__version__), end="\t")
Δt = time() - t0
print(f"\n\nΔt: {Δt: 4.1f}s.")

%matplotlib inline

**Verify that TensorFlow is finding the GPU**

In [ ]:
print("tensorflow: {0}".format(tf.__version__), end="\n\n")
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Helper functions

#### `getFeatureRepresentations()`

In [ ]:
def getFeatureRepresentations(x, layer_names, model):
    """
    Get feature representations of input x for one or more layers in a given model.
    Before invoking this, run:

       from keras import backend as K

    """
    featMatrices = []
    for ln in layer_names:
        selectedLayer = model.get_layer(ln)
        featRaw = selectedLayer.output
        featRawShape = K.shape(featRaw).eval(session=tf_session)
        N_l = featRawShape[-1]
        M_l = featRawShape[1]*featRawShape[2]
        featMatrix = K.reshape(featRaw, (M_l, N_l))
        featMatrix = K.transpose(featMatrix)
        featMatrices.append(featMatrix)
        del selectedLayer
        del featRaw
        del featRawShape
        del N_l
        del M_l
        del featMatrix
    return featMatrices

#### `getContentLoss()`

In [ ]:
def getContentLoss(F, P):
    """
    P represents the content features
    """
    cLoss = 0.5*K.sum(K.square(F - P))
    return cLoss

#### `getGramMatrix()`

The Gram matrix is pretty simple:

$$\mathbf{G} = \mathbf{F} \cdot \mathbf{F}^T$$

Alternatively, $G_{ij} = F^{ik} F_{kj}$, and this implies that $G_{ij}$ contains the dot product of column $i$ of $\mathbf{F}$ with row $j$ of $\mathbf{F}$.
Note that $\mathbf{G}$ is symmetric, and that local information about $\mathbf{F}$ is lost.

In [ ]:
def getGramMatrix(F):
    """
    F contains the style image feature matrix
    """
    G = K.dot(F, K.transpose(F))
    return G

#### `getStyleLoss()`

In [ ]:
def getStyleLoss(ws, Gs, As):
    sLoss = K.variable(0.)
    for w, G, A in zip(ws, Gs, As):
        M_l = K.int_shape(G)[1]
        N_l = K.int_shape(G)[0]
        G_gram = getGramMatrix(G)
        A_gram = getGramMatrix(A)
        sLoss += w*0.25*K.sum(K.square(G_gram - A_gram))/ (N_l**2 * M_l**2)
        del M_l
        del N_l
        del G_gram
        del A_gram
    return sLoss

#### `getTotalLoss()`

In [ ]:
# def getTotalLoss(gImPlaceholder, alpha=1.0, beta=10000.0):
def getTotalLoss(gImPlaceholder, alpha=1.0, beta=1000.0):
    F = getFeatureRepresentations(gImPlaceholder, layer_names=[contentLayerName], model=generatedModel)[0]
    Gs = getFeatureRepresentations(gImPlaceholder, layer_names=styleLayerNames, model=generatedModel)
    contentLoss = getContentLoss(F, P)
    styleLoss = getStyleLoss(ws, Gs, As)
    totalLoss = alpha*contentLoss + beta*styleLoss
    del F
    del Gs
    del contentLoss
    del styleLoss
    return totalLoss

#### `calculateLoss()`

In [ ]:
def calculateLoss(gImArr):
    """
    Calculate total loss using K.function
    """
    if gImArr.shape != (1, targetWidth, targetWidth, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    loss_fcn = K.function([generatedModel.input], [getTotalLoss(generatedModel.input)])
    return loss_fcn([gImArr])[0].astype('float64')

#### `getGrad()`

In [ ]:
def getGrad(gImArr):
    """
    Calculate the gradient of the loss function with respect to the generated image
    """
    if gImArr.shape != (1, targetWidth, targetHeight, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    grad_fcn = K.function([generatedModel.input], 
                          K.gradients(getTotalLoss(generatedModel.input), [generatedModel.input]))
    grad = grad_fcn([gImArr])[0].flatten().astype('float64')
    del grad_fcn
    return grad

#### `postProcessArray()`

In [ ]:
def postProcessArray(x):  # , rgbMean):
    # Zero-center by mean pixel
    if x.shape != (targetWidth, targetHeight, 3):
        x = x.reshape((targetWidth, targetHeight, 3))
    # I'm guessing that these hard-coded offsets are color-averages for the (train?) set for VGG16 on imagenet
    x[..., 0] += 103.939
    x[..., 1] += 116.779
    x[..., 2] += 123.68
    # 'BGR'->'RGB'
    x = x[..., ::-1]
    x = np.clip(x, 0, 255)
    x = x.astype('uint8')
    return x

#### `reprocessArray()`

In [ ]:
def reprocessArray(x):
    x = np.expand_dims(x.astype('float64'), axis=0)
    x = preprocess_input(x)
    return x

#### `saveOriginalSize()`

In [ ]:
def saveOriginalSize(x, targetSize, targetImagePath):
    xIm = Image.fromarray(x)
    xIm = xIm.resize(targetSize)
    xIm.save(targetImagePath)
    # return xIm
    return

## Load images

### Content image (a photo)

In [ ]:
targetWidth = 256
targetHeight = 256
targetSize = (targetHeight, targetWidth)

contentImagePath = './ChildrenKolkataIndiaByLorenJosephOnUnsplash395x256.png'

contentImageOrig = Image.open(contentImagePath)
contentImageSizeOrig = contentImageOrig.size
contentImage = load_img(path=contentImagePath, target_size=targetSize)
contentImageArr = img_to_array(contentImage)
contentImageArr = K.variable(preprocess_input(np.expand_dims(contentImageArr, axis=0)), dtype='float32')

### Style image (Bonnie Wilber painting)

In [ ]:
styleImagePath = './BonnieWilberFamilyFriends395x256.png'

styleImage = load_img(path=styleImagePath, target_size=targetSize)
styleImageArr = img_to_array(styleImage)
styleImageArr = K.variable(preprocess_input(np.expand_dims(styleImageArr, axis=0)), dtype='float32')

### Target image

In [ ]:
targetImage0 = np.random.randint(256, size=(targetWidth, targetHeight, 3)).astype('float64')
targetImage0 = preprocess_input(np.expand_dims(targetImage0, axis=0))
targetImage0Placeholder = K.placeholder(shape=(1, targetWidth, targetHeight, 3))

## Generate the image


### Create session, instances of VGG16

In [ ]:
tf_session = K.get_session()
contentModel = VGG16(include_top=False, weights='imagenet', input_tensor=contentImageArr)
styleModel = VGG16(include_top=False, weights='imagenet', input_tensor=styleImageArr)
print("\nstyle.Model.summary():\n", styleModel.summary())
generatedModel = VGG16(include_top=False, weights='imagenet', input_tensor=targetImage0Placeholder)

#### Specify mutable layer names

In [ ]:
contentLayerName = 'block4_conv2'
styleLayerNames = [
                   'block1_conv1',
                   'block2_conv1',
                   'block3_conv1',
                   'block4_conv1',
                  ]

#### Get feature weights

In [ ]:
P = getFeatureRepresentations(x=contentImageArr, layer_names=[contentLayerName], model=contentModel)[0]
As = getFeatureRepresentations(x=styleImageArr, layer_names=styleLayerNames, model=styleModel)
ws = np.ones(len(styleLayerNames))/float(len(styleLayerNames))

### Generate the target images

#### Show target images at 20 times

In [ ]:
exps = np.array(range(2, 22))
b = np.exp(np.log(600)/21)
b
printIterations = [int(b**e) for e in exps]
# printIterations.insert(0, 0)
print(printIterations)

In [ ]:
# printIterations = printIterations[13:]
# print(printIterations)

##### Set random seeds to enforce consistent results

This is described in more detail in Jason Brownlee's [How to Get Reproducible Results with Keras](https://machinelearningmastery.com/reproducible-results-neural-networks-keras/)

**Note: this will not necessarily work when running on GPUs.**

2 Steps:
1. Set `numpy`'s random number generator seed, since Keras itself relies on this
1. Set TensorFlow's random number generator seed, as this is invoked by TF's random number generators.

In [ ]:
from numpy.random import seed
seed(3)

from tensorflow import set_random_seed
set_random_seed(4)

##### initialize target

In [ ]:
print(asctime(localtime()))
t0 = time()
x = targetImage0

iterations = 600

# Print out initial state of random generated image
tlast = t0
targetImagePath = f"./test-395x256at{0:03d}.jpg"
xOut = postProcessArray(x.copy())
xIm = saveOriginalSize(xOut, contentImageSizeOrig, targetImagePath)

for i in range(1, iterations + 1):
    if i > 1:
        print(f"{i:03d}\t{asctime(localtime())[11:19]}", end='')
    x, minVal, info = fmin_l_bfgs_b(calculateLoss, x.flatten(),
                                    fprime=getGrad, maxfun=10, maxiter=3)
#                                     fprime=getGrad, maxiter=1)
    ti = time()
    Δt = ti - tlast
    Δti0 = ti - t0
    if i == 1:
        print("\n  i\t       t\t\t     Δt\t\t   Δti0\t\t    loss   funcalls     n_iter\t\t   image")
        print(f"{i:03d}\t{asctime(localtime())[11:19]}", end='')
#     print(int(Δt//3600), int((Δt % 3600)//60), Δt % 60.0, int(Δti0//3600),
#           int((Δti0 % 3600)//60), Δti0 % 60.0, minVal,
#           info['funcalls'], info['nit'])
    print(f"\t{int(Δt//3600):2d}h, {int((Δt % 3600)//60):2d}m,"
          f" {Δt % 60.0:4.1f}s\t{int(Δti0//3600):2d}h,"
          f" {int((Δti0 % 3600)//60):2d}m,"
          f" {Δti0 % 60.0:4.1f}s\t{minVal:015.2f}"
          f"\t{info['funcalls']:3d}\t{info['nit']:3d}", end='')
    # save current generated image
    if i in printIterations:
        targetImagePath = f"./test395x256at{i:03d}.jpg"
        xOut = postProcessArray(x.copy())
        saveOriginalSize(xOut, contentImageSizeOrig, targetImagePath)
        print(f"\t{targetImagePath}")
    else:
        print('') 
    tlast = ti

print("Done")